In [1]:
! pip install -q litellm tqdm PyMuPDF Pillow instructor openai python-dotenv weaviate-client pandas

In [2]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [22]:
def get_singleqa_search_o1_instruction(MAX_SEARCH_LIMIT):
    return (
        "You are a planning agent in a retrieval-augmented generation system. "
        "Your job is to create a plan for answering the question using external tools.\n\n"
        "Do not try to answer the question directly.\n"
        "Instead, generate a plan with search queries to retrieve relevant information.\n\n"
        "To specify a search query, write it between these tags:\n"
        "<|begin_search_query|>your query here<|end_search_query|>\n\n"
        f"You can specify up to {MAX_SEARCH_LIMIT} search queries.\n\n"
        "After generating the plan, the system will execute the search queries and provide you with the results.\n" 
        "You will then have the opportunity to use those results to answer the question."
    )

In [23]:
print(get_singleqa_search_o1_instruction(5)) #get_singleqa_search_o1_instruction(5)

You are a planning agent in a retrieval-augmented generation system. Your job is to create a plan for answering the question using external tools.

Do not try to answer the question directly.
Instead, generate a plan with search queries to retrieve relevant information.

To specify a search query, write it between these tags:
<|begin_search_query|>your query here<|end_search_query|>

You can specify up to 5 search queries.

After generating the plan, the system will execute the search queries and provide you with the results.
You will then have the opportunity to use those results to answer the question.


In [30]:
def get_task_instruction_openqa(question, model_name=None):
    if model_name == 'qwq':
        user_prompt = (
            'Please Provide the Plan for the following question.\n\n'
            f'Question:\n{question}\n\n'
        )
    else:
        user_prompt = (
            'Please answer the following question. You should think step by step to solve it.\n\n'
            'Provide your final answer in the format \\boxed{YOUR_ANSWER}.\n\n'
            f'Question:\n{question}\n\n'
        )
    return user_prompt

In [31]:
question = """
Is 3M a capital-intensive business based on FY2022 data?
"""

In [32]:

user_prompt = get_task_instruction_openqa(question,'qwq')
instruction = get_singleqa_search_o1_instruction(5)

prompt = [{"role": "user", "content": instruction + user_prompt}]

In [33]:
prompt

[{'role': 'user',
  'content': 'You are a planning agent in a retrieval-augmented generation system. Your job is to create a plan for answering the question using external tools.\n\nDo not try to answer the question directly.\nInstead, generate a plan with search queries to retrieve relevant information.\n\nTo specify a search query, write it between these tags:\n<|begin_search_query|>your query here<|end_search_query|>\n\nYou can specify up to 5 search queries.\n\nAfter generating the plan, the system will execute the search queries and provide you with the results.\nYou will then have the opportunity to use those results to answer the question.Please Provide the Plan for the following question.\n\nQuestion:\n\nIs 3M a capital-intensive business based on FY2022 data?\n\n\n'}]

In [34]:
from litellm import completion
response = completion(
        model="openrouter/qwen/qwq-32b-preview",
        messages=prompt,
        temperature=0.2
    )
print(response)

ModelResponse(id='gen-1737428086-aPPamgrYKq25bNqhFkpE', created=1737428086, model='openrouter/qwen/qwq-32b-preview', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Alright, I need to figure out if 3M is a capital-intensive business based on their FY2022 data. First off, I need to understand what a capital-intensive business is. I think it means a company that requires a lot of initial investment in physical assets like property, plant, and equipment to operate. So, for 3M, which is a multinational corporation that operates in various industries like manufacturing, health care, consumer goods, and more, I need to see how much they invest in assets relative to their sales or revenue.\n\nI recall that in finance, there are ratios that can help determine how capital-intensive a company is, like the fixed assets to sales ratio or the capital intensity ratio. Maybe I should look into those. I need to find 3M\'s fina

In [35]:
print(response.choices[0].message.content)

Alright, I need to figure out if 3M is a capital-intensive business based on their FY2022 data. First off, I need to understand what a capital-intensive business is. I think it means a company that requires a lot of initial investment in physical assets like property, plant, and equipment to operate. So, for 3M, which is a multinational corporation that operates in various industries like manufacturing, health care, consumer goods, and more, I need to see how much they invest in assets relative to their sales or revenue.

I recall that in finance, there are ratios that can help determine how capital-intensive a company is, like the fixed assets to sales ratio or the capital intensity ratio. Maybe I should look into those. I need to find 3M's financial statements for FY2022 to get the necessary data. Specifically, I'll need their balance sheet and income statement to find the total assets and sales figures.

I should probably start by finding 3M's annual report for FY2022. Annual report

In [48]:
# Extract the search query with <|begin_search_query|> and <|end_search_query|>
search_query = response.choices[0].message.content.split("<|begin_search_query|>")[1].split("<|end_search_query|>")[0].strip()
print("Search Query:", search_query)



Search Query: is 3M a capital intensive business 2022
